# Module 8 - Regression

Today's data set consists of time series data measured at 10 min intervals for about 4.5 months. A house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions every 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). We will use this data to predict the current energy usage of the appliances in the home based on the indoor and outdoor predictors. Notice, this is not forecasting. 

## Setup
Let's get all the requirements sorted before we move on to the excercise. Notice, today we will be using kaggle to manage the data download. We'll discuss this in more detail in the data download section.  

In [ ]:
# Requirements
!pip install --upgrade ipykernel
!pip install kaggle
!pip install datetime
!pip install pandas
!pip install tableone
!pip install numpy
!pip install matplotlib
!pip install scipy
!pip install boruta
!pip install sklearn

# Globals
seed = 1017

#imports
import kaggle
import pandas as pd
from datetime import datetime
from tableone import TableOne
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

#magic
%matplotlib inline

## Loading the data
Today we will be downloading data directly from [kaggle](kaggle.com). Do do this you will need to have a kaggle account. **It's free.** Once you create your kaggle account you can generate an API token. After you log in you should see a circular account icon in the upper-right of any kaggle page. Clicking on your account icon will open a right-sidebar where you can select "Account" to edit your account. Scroll down to the API section and click on the "create new api token" button. An API token should automatically download and a prompt will also appear telling you which directory to put this token so python knows where find it. For MacOS users this location is "~/.kaggle/kaggle.json".

In [ ]:
#log in to kaggle using your api token
kaggle.api.authenticate()

#path relative to this notebook to put the data
datadir = 'data'

#name of the dataset on kaggle
dataset = 'loveall/appliances-energy-prediction'

#downlaod the data
kaggle.api.dataset_download_files(dataset, path=datadir, unzip=True)

In [ ]:
# download the data as a pandas dataframe
df = pd.read_csv("data/KAG_energydata_complete.csv")

## Data description
|column | description |
|-------|-------------|
|date | time year-month-day hour:minute:second|
|Appliances | energy use in Wh |
|lights| energy use of light fixtures in the house in Wh |
|T1| Temperature in kitchen area, in Celsius|
|RH1| Relative humidity in kitchen area, in %|
|T2| Temperature in living room area, in Celsius |
|RH2| Relative humidity in living room area, in % |
|T3| Temperature in laundry room area, in Celsius |
|RH3| Humidity in laundry room area, in % |
|T4| Temperature in office room, in Celsius|
|RH4| Humidity in office room, in %|
|T5| Temperature in bathroom, in Celsius|
|RH5| Humidity in bathroom, in % |
|T6| Temperature outside the building (north side), in Celsius |
|RH6| Humidity outside the building (north side), in %|
|T7| Temperature in ironing room, in Celsius|
|RH7| Humidity in ironing room, in % |
|T8| Temperature in teenager room 2, in Celsius |
|RH8| Humidity in teenager room 2, in %|
|T9| Temperature in parents room, in Celsius|
|RH9| Humidity in parents room, in % |
|Tout| Temperature outside (from Chievres weather station), in Celsius|
|Press_mm_hg| Barometric Pressure at Chievres weather station, in mm Hg |
|RHout| Humidity outside (from Chievres weather station), in %|
|Windspeed| Wind speed at Chievres weather station, in m/s|
|Visibility| Ground visibility at Chievres weather station, in km|
|Tdewpoint| Condensation Temperature at Chievres weather station, in Celsius|
|rv1| Random variable 1, nondimensional|
|rv2| Random variable 2, nondimensional|

## Formatting
As is, the date column is acting like a bookeeping index for each observation. Maybe we can get some useful perdictors out of it. Let's engineer a time of day numeric feauture and a weekend/weekday binary variable.

In [ ]:
#convert strings in date column into datetime objects
datetimelist = [ datetime.strptime(date, "%Y-%m-%d %H:%M:%S") for date in df['date'] ]

#extract time of day in minutes
df['time'] = [ obs.hour * 24.0 + obs.minute for obs in datetimelist ]

#extract the day of week as integer 0-6 for Monday-Sunday
df['day'] = [ obs.weekday() for obs in datetimelist ]

#remove the date column
df.drop('date', axis=1, inplace=True)

In [ ]:
#Bin appliance energy usage into above and below the median to comapre features
df['bin'] = df['Appliances']>np.median(df['Appliances'])

# Generate table 1 - group by the Appliance energy use bins
tbl1 = TableOne(df, groupby='bin', categorical=['day', 'lights'], 
                pval=True,
                dip_test=True,
                normal_test=True,
                tukey_test=True)

#Remove the bin variable we created for diagnostic puroposes
df.drop('bin', axis=1, inplace=True)

In [ ]:
#display the table 1
display(tbl1)

### Feature distributions
Let's plot the feature distributions and see if we can address the warnings raised by the table 1.

In [ ]:
#plot the feature distributions
for feat in df.columns: 
    df[[feat]].dropna().plot.kde(bw_method='scott') #use bw_method=.02 for a lower bandwidth gaussian representation
    plt.legend([feat])
    plt.show()

In [ ]:
#Impute any missing values with their column median
df.fillna(value=df.median(axis=1, skipna=True), inplace=True)

In [ ]:
#log2 transform - you will need to identify any features with long tails
cols = ['Appliances', 'lights']
df[cols] = np.log(df[cols]+1)

In [ ]:
#mean center and unit scale the standard deviation for the numerical variables
cols = df.columns[~df.columns.isin(['day','lights'])]
df[cols] = stats.zscore(df[cols])

In [ ]:
#70-30 partition
df_test = df.sample(frac=0.3)
df_train = df.drop(df_test.index)
display(df_train.shape)
display(df_test.shape)

## Linear Regression 
Let's explore linear regression with regularization using the ridge, lasso, and elasticnt regression models.

In [ ]:
from sklearn.linear_model import RidgeCV

# get predictors and labels
X = np.array(df_train.drop('Appliances', axis=1)) 
y = np.array(df_train['Appliances'])


#train train ridge regression model with 10-fold cross validataion
ridge = RidgeCV(cv=5).fit(X,y)

#plot feature importance based on coeficients
importance = np.abs(ridge.coef_)
feature_names = np.array(df.columns.drop('Appliances'))
plt.bar(height=importance, x=feature_names)
plt.xticks(rotation=90)
plt.title("Feature importances via coefficients")
plt.show()

# Get model score in the testing set
X_test = np.array(df_test.drop('Appliances', axis=1)) 
y_test = np.array(df_test['Appliances'])

#display the model score
display(ridge.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LassoCV

# get predictors and labels
X = np.array(df_train.drop('Appliances', axis=1)) 
y = np.array(df_train['Appliances'])

#train lasso model with 10-fold cross validataion
lasso = LassoCV(max_iter=1000, tol=0.0001, cv=5,
                verbose=0, n_jobs=-1, random_state=seed, selection='random').fit(X,y)

#display the model score
display(lasso.score(X, y))

#plot feature importance based on coeficients
importance = np.abs(lasso.coef_)
feature_names = np.array(df.columns.drop('Appliances'))
plt.bar(height=importance, x=feature_names)
plt.xticks(rotation=90)
plt.title("Feature importances via coefficients")
plt.show()

# Get model score in the testing set
X_test = np.array(df_test.drop('Appliances', axis=1)) 
y_test = np.array(df_test['Appliances'])

#display the model score
display(lasso.score(X_test, y_test))


In [ ]:
from sklearn.linear_model import ElasticNetCV

# get predictors and labels
X = np.array(df_train.drop('Appliances', axis=1)) 
y = np.array(df_train['Appliances'])

#train lasso model with 10-fold cross validataion
elastic = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=1000, tol=0.0001, cv=5,
                     verbose=0, n_jobs=-1, random_state=seed, selection='random').fit(X,y)

#display the model score
display(elastic.score(X, y))

#plot feature importance based on coeficients
importance = np.abs(elastic.coef_)
feature_names = np.array(df.columns.drop('Appliances'))
plt.bar(height=importance, x=feature_names)
plt.xticks(rotation=90)
plt.title("Feature importances via coefficients")
plt.show()

# Get model score in the testing set
X_test = np.array(df_test.drop('Appliances', axis=1)) 
y_test = np.array(df_test['Appliances'])

#display the model score
display(elastic.score(X_test, y_test))

## Non-Linear Regression
Let's explore non-linear regression with a multi layer perceptron model

In [ ]:
from sklearn.neural_network import MLPRegressor

# get predictors and labels
X = np.array(df_train.drop('Appliances', axis=1)) 
y = np.array(df_train['Appliances'])

#train AdaBoost model 
mlp = MLPRegressor(hidden_layer_sizes=(10,10), activation='relu',
                        solver='adam', alpha=0.0001, batch_size='auto',
                        learning_rate='constant', learning_rate_init=0.001,
                        max_iter=200, shuffle=True, random_state=seed, tol=0.0001,
                        verbose=False, warm_start=False, 
                        early_stopping=True, validation_fraction=0.1,
                        beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10).fit(X,y)

# Get model score in the testing set
X_test = np.array(df_test.drop('Appliances', axis=1)) 
y_test = np.array(df_test['Appliances'])

#display the model score
display(mlp.score(X_test, y_test))
